# <font color=red>HW01-q2 Vision Course, Perspective</font>
This is the notebook for **q2.py** that is the implementation of **Perspective**. <br>
The code is written by **Asal Mehradfar** with student number **96105434**.

## <font color=orange>Description</font>
I used the course slides for implementing this part. the brief explanation:


*   In this question, I work with colorful images. So at the first step I convert the image to the RGB one. <br><br>
*   Then I need to compute the four important parameters in the formula of **Homography**, $K'$, $K^{-1}$, $R$ and $t$. The number of camera pixels and the distance and height of the camera are given. $P_x$ and $P_y$ are chosen as the center of the field according to class points. Also s is defined to be zero. Now we can easily compute $H$ and $H^{-1}$. <br>
$$K' = K =\begin{bmatrix} f & 0 & P_x \\ 0 & f & P_y \\ 0 & 0 & 1 \end{bmatrix} = \begin{bmatrix} 500 & 0 & 128 \\ 0 & 500 & 128 \\ 0 & 0 & 1 \end{bmatrix}$$
$$K^{-1} =\begin{bmatrix} 0.002 & 0 & -0.256 \\ 0 & 0.002 & -0.256 \\ 0 & 0 & 1 \end{bmatrix}$$
$$\Phi = -\arctan(\frac{d}{h})$$
$$R =\begin{bmatrix} \cos(\Phi) & 0 & -\sin(\Phi) \\ 0 & 1 & 0 \\ \sin(\Phi) & 0 & \cos(\Phi) \end{bmatrix} = \begin{bmatrix} 0.53 & 0 & 0.85 \\ 0 & 1 & 0 \\ -0.85 & 0 & 0.53 \end{bmatrix}$$
$$t =R \times C = R \times \begin{bmatrix} d \\ 0 \\ 0 \end{bmatrix} = \begin{bmatrix} -21.2 \\ 0 \\ 33.92 \end{bmatrix}$$
*   $$H = K'(R - \frac{tn^{t}}{d})K^{-1}$$
$$n = \begin{bmatrix} 0 \\ 0 \\ -1 \end{bmatrix}$$
$$H =\begin{bmatrix} 0.313 & 0 & 201.457 \\ -0.217 & 1 & 141.297 \\ -0.002 & 0 & 2.104 \end{bmatrix}$$
$$H^{-1} =\begin{bmatrix} 2.104 & 0 & -201.457 \\ 0.217 & 1 & -87.947 \\ 0.002 & 0 & 0.313 \end{bmatrix}$$
<br>
*   At the last step, I find the size of the output image by finding the maximum and minimum of the output coordinates using $H^{-1}$. Then I make a zero matrix of that size and convert all the coordinates of the output image to the coordinates of the input logo and use the first image's values for them.










### <font color=yellow>Imports</font>

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math
from PIL import Image

### <font color=yellow>Parameters</font>


*   D = 40 
*   H = 25
*   N = 500


In [2]:
D = 40
H = 25
N = 500
n = np.array([[0, 0, -1]]) # computed and tested

### <font color=yellow>Functions</font>

In [3]:
def get_img(path):
    """
    Read the image file from the path and change it from BGR to RGB
    pay attention that in open-cv colorful images are BGR **NOT** RGB
    
    Inputs:
    --> path: path for the image
    Outputs:
    ==> img: the RGB image
    """
    img = cv2.imread(path)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    return img

def plot_img(img, path=None):
    """
    Plot a colorful image and save it if needed
    
    Inputs:
    --> img: the desired image
    --> path: the default value is None, if it is given the image will be saved in the path
    Outputs:
    ==> Nothing, the image will be plotted
    """
    fig = plt.figure(figsize=(16, 8))
    plt.imshow(img)
    plt.axis('off')
    if path != None:
        fig.savefig(path, bbox_inches='tight')
    plt.show()

def get_params(f, Px, Py, d, h, s=0):
    """
    compute the parameters needed for making K, K^(-1), R, t
    
    Inputs:
    --> f: number of camera pixels
    --> Px: x center of image
    --> Py: y center of image
    --> d: distance between camera and center of the football field
    --> h: height of the camera from the earth
    --> s: is usually zero or near zero for creating K
    Outputs:
    ==> K: here is equal to K' in the formula
    ==> K_inv: K^(-1)
    ==> R
    ==> t
    """
    K = np.array([[f, s, Px], [0, f, Py], [0, 0, 1]])
    K_inv = np.linalg.inv(K)
    angle = -np.arctan(d / h)
    R = np.array([[np.cos(angle), 0, -np.sin(angle)], [0, 1, 0], [np.sin(angle), 0, np.cos(angle)]])
    C = np.array([[d, 0, 0]]).transpose()
    t = -np.dot(R, C)
    return K, K_inv, R, t

def get_H(K, K_inv, R, t, n, d):
    """
    compute H and H_inv
    
    Inputs:
    --> K: 3*3 array, here is equal to K' in the formula
    --> K_inv: K^(-1)
    --> R: 3*3 array 
    --> t: 3*1 vector
    --> n: a vector which is vertical to the plane, being negative was understood by experiment
    --> d: distance between camera and center of the football field
    Outputs:
    ==> H: the homography 3*3 array computed by the formula in page 15 of slide 8
    ==> H_inv: H^(-1)
    """
    H = np.dot(np.dot(K, R - np.dot(t, n) / d), K_inv)
    H_inv = np.linalg.inv(H)
    return H, H_inv

def make_new_img(H, H_inv, img):
    """
    here at first we try to find the size of the output image 
    by computing the effect of H inverse on the points in the input image 
    and finding their maximum and minimum. 
    then by making a completely zero 3d array of the new size we use H 
    for computing new coordinates in the original image and used the values of the original image 
    for setting the values of new image.
    
    Inputs:
    --> H: the homography 3*3 array computed by the formula in page 15 of slide 8
    --> H_inv: H^(-1)
    --> img: the original image, here the original logo
    Outputs:
    ==> new_img: the output image after homography
    """
    min_x = math.inf
    min_y = math.inf
    max_x = 0
    max_y = 0
    for x in range(img.shape[0]):
        for y in range(img.shape[1]):
            a = np.array([[x, y, 1]]).transpose()
            b = np.dot(H_inv, a)
            [x_new, y_new] = [int(b[0]/b[2]), int(b[1]/b[2])]
            min_x = min(x_new, min_x)
            min_y = min(y_new, min_y)
            max_x = max(x_new, max_x)
            max_y = max(y_new, max_y)

    new_img = np.zeros((max_x - min_x + 1, max_y - min_y + 1, 3))
    for x in range(new_img.shape[0]):
        for y in range(new_img.shape[1]):
            a = np.array([[x + min_x, y + min_y, 1]]).transpose()
            b = np.dot(H, a)
            [x_new, y_new] = [int(b[0]/b[2]), int(b[1]/b[2])]
            if 0 < x_new < img.shape[0] and 0 < y_new < img.shape[1]:
                new_img[x, y, :] = img[x_new, y_new, :]
    new_img = new_img.astype(np.uint8)
    return new_img

def save_img(array, path, scaled=False):
    """
    save the input image in the desired path

    Inputs:
    --> array: the array of an image
    --> path: the desired path for saving the image
    --> scaled: the default value is False,
    if it is given True the image will be scaled into [0,255]
    Outputs:
    ==> Nothing, just saving the image
    """
    if scaled:
        array = scaling_img(array).astype(np.uint8)
    img = Image.fromarray(array)
    img.save(path)

def scaling_img(img):
    """
    return the scaled image usually for saving 
    
    Inputs:
    --> img: the desired image for scaling
    Outputs:
    ==> scaled_img: we assume that the minimum of the image is zero
        so we scale it by devision by its maximum and multiplying it by 255.0
    """
    scaled_img = 255.0 * img / np.max(img)
    return scaled_img


### <font color=yellow>Main Part, Test on logo.png</font>

In [4]:
img = get_img('logo.png')
K, K_inv, R, t = get_params(N, img.shape[0]/2, img.shape[0]/2, D, H)
H , H_inv = get_H(K, K_inv, R, t, n, H)
new_img = make_new_img(H, H_inv, img)
save_img(new_img, 'res12.jpg')